In [1]:
%load_ext autoreload  
%autoreload 2 
import datasets
from subset_active_learning.subset_selection import select, preprocess, compare
import wandb
import numpy as np
import torch

/home/glai/anaconda3/envs/subset/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def random_select_subset(ds, seed, size):
    np.random.seed(seed)
    selected_indices = np.random.choice(len(ds), replace=False, size=size)
    return ds.select(selected_indices)

In [3]:
DATA_SEED = 10
MODEL_SEED = 2
TRAIN_SIZE = 500
MODEL_CARD = "roberta-base"
processed_ds = preprocess.preprocess_sst2(MODEL_CARD)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 753.15it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e96fff4bcf800b80.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c19c9b6a9c9f8182.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8d2a9f8a95240b96.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0c3dbb722d1f6005.arrow
Loading cache

In [4]:
comparison_run = compare.ComparisonRun(train_ds = random_select_subset(ds=processed_ds["train"], seed=DATA_SEED, size=TRAIN_SIZE), \
                                                valid_ds = processed_ds["validation"], \
                                                test_ds = processed_ds["test"], \
                                                data_seed = DATA_SEED,
                                                model_seed = MODEL_SEED)

In [5]:
config = select.SubsetTrainingArguments(model_card=MODEL_CARD, max_steps=500); config

SubsetTrainingArguments(model_card='roberta-base', pretraining=True, max_steps=500, eval_steps=500, learning_rate=1e-05, batch_size=8, adam_epsilon=1e-06, adam_beta1=0.9, adam_beta2=0.999, max_grad_norm=1.0, warmup_ratio=0.1, weight_decay=0.01, num_labels=2, metric='accuracy')

In [6]:
comparison_run.one_run(wandb_tags=[MODEL_CARD, "debug-roberta", f"train_size-{TRAIN_SIZE}", "new_param"], config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garylai (johnny-gary). Use `wandb login --relogin` to force relogin


##### Warning: Hard Setting Model Seed to 2


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

batch_size,▁
loss,████▇▇▄▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▄█▄▁
batch_size,8
loss,8e-05
run_time,230.57
sst2_final_test:accuracy,0.83348
sst2_final_valid:accuracy,0.82561


In [6]:
comparison_run.data_seed

10